<a href="https://colab.research.google.com/github/walexand3r/4YP/blob/master/CNN_for_FashionMNIST_v2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Model Version 2.3

Simple sequential Keras Model for use on Fashion MNIST dataset

Carries out four tests

Added: 
 - Merged training and prediction steps
 - Duplicated entire process for each test combination

To Add:
 - Turn Test blocks into a function, for now, works fine as all blocks run sequentially




In [0]:

# This block imports relevant packages and sets intial seed to ensure reproducability
# Seed value
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os, datetime
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
%tensorflow_version 2.x
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: tf.random.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
import tensorflow.keras as keras
# from tensorflow.keras import backend as K
# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

%load_ext tensorboard
import pandas as pd

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import numpy as np

TensorFlow 2.x selected.


In [0]:
# Mount drive

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
num_classes = 10    
batch_size = 128
epochs = 50

# Build and compile model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(learning_rate = 1),
              metrics=['accuracy'])
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [0]:
# Sets paths for train/test D/D+
folder = 'Intensity Scaled'
path = '/content/drive/My Drive/4th Year/4YP/Fashion MNIST 60k/Processed Datasets/'
pathDtrain  = path[:74] + folder  + '/D_train.csv'
pathDplustrain  = path[:74] + folder  + '/D+_train.csv'
pathDtest  = path[:74] + folder  + '/D_test.csv'
pathDplustest  = path[:74] + folder  + '/D+_test.csv'

In [0]:
# Test 1: R(M,D)     
# Full data import, learning and test of: D_train and D_test
# loading modified Fashion MNIST dataset
train = pd.read_csv(pathDtrain,header=None)
test = pd.read_csv(pathDtest,header=None)
(n_train,pixels) = np.shape(train)
n_test = np.size(test,0)
dim = int(np.sqrt(pixels-1))

# Getting data in right form
# input image dimensions
img_rows, img_cols = dim, dim

# Extracting data
test_labels = test.iloc[:,0]
test_values = test.iloc[:,1:]
train_labels = train.iloc[:,0]
train_values = train.iloc[:,1:]
x_train = train_values.values.reshape(n_train,img_rows,img_cols,1)
x_test = test_values.values.reshape(n_test,img_rows,img_cols,1)

# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(train_labels, num_classes)
y_test = keras.utils.to_categorical(test_labels, num_classes)

# Fit model and learn
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

# Make predictions
predictions_1 = model.predict(x_test, batch_size=batch_size,
                            verbose=1,)
# Forming into required raw data format
predictions_1 = predictions_1.round(0)
MLresults_1 = np.argmax(predictions_1,axis=1)

Correctness_1 = np.zeros(len(predictions_1))
for i in range(len(predictions_1)):
  if np.array_equal(MLresults_1[i],test_labels[i]):
    Correctness_1[i] = True
  else:
    Correctness_1[i] = False

GroundTruth_1 = test_labels

RawData_1 = np.zeros((n_test,5))
RawData_1[:,1] = GroundTruth_1
RawData_1[:,2] = MLresults_1
RawData_1[:,4] = Correctness_1

for i in range(len(GroundTruth_1)):
  RawData_1[i,0] = i+1

# Saving
savePath = '/content/drive/My Drive/4th Year/4YP/Fashion MNIST 60k/Results/' + folder + '/R(M,D).csv'
#pd.DataFrame(RawData_1).to_csv(savePath,header=None, index=None)

# # Testing accuracy for interest
# sum(Correctness_1) / 10000
# #y_test = y_test.astype(np.float32)
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test accuracy:', score[1])



Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 5s 89us/sample - loss: 2.2780 - accuracy: 0.5596 - val_loss: 0.4938 - val_accuracy: 0.8124
Epoch 2/50
60000/60000 [==============================] - 5s 81us/sample - loss: 0.5275 - accuracy: 0.8110 - val_loss: 0.4252 - val_accuracy: 0.8403
Epoch 3/50
60000/60000 [==============================] - 5s 82us/sample - loss: 0.4520 - accuracy: 0.8370 - val_loss: 0.3884 - val_accuracy: 0.8583
Epoch 4/50
60000/60000 [==============================] - 5s 81us/sample - loss: 0.4185 - accuracy: 0.8503 - val_loss: 0.3695 - val_accuracy: 0.8610
Epoch 5/50
60000/60000 [==============================] - 5s 82us/sample - loss: 0.3941 - accuracy: 0.8579 - val_loss: 0.3340 - val_accuracy: 0.8723
Epoch 6/50
60000/60000 [==============================] - 5s 80us/sample - loss: 0.3741 - accuracy: 0.8659 - val_loss: 0.3263 - val_accuracy: 0.8794
Epoch 7/50
60000/60000 [==============================] 

In [0]:
savePath = '/content/drive/My Drive/4th Year/4YP/Fashion MNIST 60k/Results/' + folder + '/R(M,D).csv'
#pd.DataFrame(RawData_1).to_csv(savePath,header=None, index=None)
print('R(M,D+):',(sum(Correctness_1) / 10000))

R(M,D+): 0.8934


In [0]:
# Test 2: R(M+,D)
# Full data import, learning and test of: D_train and D_test
# loading modified Fashion MNIST dataset
train = pd.read_csv(pathDplustrain,header=None)
test = pd.read_csv(pathDtest,header=None)
(n_train,pixels) = np.shape(train)
n_test = np.size(test,0)
dim = int(np.sqrt(pixels-1))

# Getting data in right form
# input image dimensions
img_rows, img_cols = dim, dim

# Extracting data
test_labels = test.iloc[:,0]
test_values = test.iloc[:,1:]
train_labels = train.iloc[:,0]
train_values = train.iloc[:,1:]
x_train = train_values.values.reshape(n_train,img_rows,img_cols,1)
x_test = test_values.values.reshape(n_test,img_rows,img_cols,1)

# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(train_labels, num_classes)
y_test = keras.utils.to_categorical(test_labels, num_classes)

# Fit model and learn
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

# Make predictions
predictions_2 = model.predict(x_test, batch_size=batch_size,
                            verbose=1,)
# Forming into required raw data format
predictions_2 = predictions_2.round(0)
MLresults_2 = np.argmax(predictions_2,axis=1)

Correctness_2 = np.zeros(len(predictions_2))
for i in range(len(predictions_2)):
  if np.array_equal(MLresults_2[i],test_labels[i]):
    Correctness_2[i] = True
  else:
    Correctness_2[i] = False

GroundTruth_2 = test_labels

RawData_2 = np.zeros((n_test,5))
RawData_2[:,1] = GroundTruth_2
RawData_2[:,2] = MLresults_2
RawData_2[:,4] = Correctness_2

for i in range(len(GroundTruth_2)):
  RawData_2[i,0] = i+1

# Saving
savePath = '/content/drive/My Drive/4th Year/4YP/Fashion MNIST 60k/Results/' + folder + '/R(M+,D).csv'
#pd.DataFrame(RawData_2).to_csv(savePath,header=None, index=None)

# # Testing accuracy for interest
# sum(Correctness_2) / 10000
# #y_test = y_test.astype(np.float32)
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test accuracy:', score[1])



Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 5s 90us/sample - loss: 1.8289 - accuracy: 0.6650 - val_loss: 1.1503 - val_accuracy: 0.6182
Epoch 2/50
60000/60000 [==============================] - 5s 81us/sample - loss: 0.5149 - accuracy: 0.8158 - val_loss: 1.3278 - val_accuracy: 0.5972
Epoch 3/50
60000/60000 [==============================] - 5s 82us/sample - loss: 0.4538 - accuracy: 0.8370 - val_loss: 2.2438 - val_accuracy: 0.5165
Epoch 4/50
60000/60000 [==============================] - 5s 82us/sample - loss: 0.4145 - accuracy: 0.8527 - val_loss: 1.2415 - val_accuracy: 0.6378
Epoch 5/50
60000/60000 [==============================] - 5s 82us/sample - loss: 0.3936 - accuracy: 0.8587 - val_loss: 1.0059 - val_accuracy: 0.6684
Epoch 6/50
60000/60000 [==============================] - 5s 82us/sample - loss: 0.3745 - accuracy: 0.8653 - val_loss: 1.1048 - val_accuracy: 0.6250
Epoch 7/50
60000/60000 [==============================] 

In [0]:
savePath = '/content/drive/My Drive/4th Year/4YP/Fashion MNIST 60k/Results/' + folder + '/R(M+,D).csv'
#pd.DataFrame(RawData_2).to_csv(savePath,header=None, index=None)
print('R(M,D+):',(sum(Correctness_2) / 10000))

R(M,D+): 0.5243


In [0]:
# Test 3: R(M,D+)
# Full data import, learning and test of: D_train and D_test
# loading modified Fashion MNIST dataset
train = pd.read_csv(pathDtrain,header=None)
test = pd.read_csv(pathDplustest,header=None)
(n_train,pixels) = np.shape(train)
n_test = np.size(test,0)
dim = int(np.sqrt(pixels-1))

# Getting data in right form
# input image dimensions
img_rows, img_cols = dim, dim

# Extracting data
test_labels = test.iloc[:,0]
test_values = test.iloc[:,1:]
train_labels = train.iloc[:,0]
train_values = train.iloc[:,1:]
x_train = train_values.values.reshape(n_train,img_rows,img_cols,1)
x_test = test_values.values.reshape(n_test,img_rows,img_cols,1)

# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(train_labels, num_classes)
y_test = keras.utils.to_categorical(test_labels, num_classes)

# Fit model and learn
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

# Make predictions
predictions_3 = model.predict(x_test, batch_size=batch_size,
                            verbose=1,)
# Forming into required raw data format
predictions_3 = predictions_3.round(0)
MLresults_3 = np.argmax(predictions_3,axis=1)

Correctness_3 = np.zeros(len(predictions_3))
for i in range(len(predictions_3)):
  if np.array_equal(MLresults_3[i],test_labels[i]):
    Correctness_3[i] = True
  else:
    Correctness_3[i] = False

GroundTruth_3 = test_labels

RawData_3 = np.zeros((n_test,5))
RawData_3[:,1] = GroundTruth_3
RawData_3[:,2] = MLresults_3
RawData_3[:,4] = Correctness_3

for i in range(len(GroundTruth_3)):
  RawData_3[i,0] = i+1

# Saving
savePath = '/content/drive/My Drive/4th Year/4YP/Fashion MNIST 60k/Results/' + folder + '/R(M,D+).csv'
#pd.DataFrame(RawData_3).to_csv(savePath,header=None, index=None)

# # Testing accuracy for interest
# sum(Correctness_3) / 10000
# #y_test = y_test.astype(np.float32)
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test accuracy:', score[1])



Train on 60000 samples, validate on 10000 samples
Epoch 1/40
60000/60000 [==============================] - 5s 90us/sample - loss: 2.1951 - accuracy: 0.4722 - val_loss: 0.5124 - val_accuracy: 0.8131
Epoch 2/40
60000/60000 [==============================] - 5s 82us/sample - loss: 0.5438 - accuracy: 0.8019 - val_loss: 0.4087 - val_accuracy: 0.8544
Epoch 3/40
60000/60000 [==============================] - 5s 82us/sample - loss: 0.4674 - accuracy: 0.8318 - val_loss: 0.3772 - val_accuracy: 0.8622
Epoch 4/40
60000/60000 [==============================] - 5s 82us/sample - loss: 0.4242 - accuracy: 0.8468 - val_loss: 0.3428 - val_accuracy: 0.8723
Epoch 5/40
60000/60000 [==============================] - 5s 81us/sample - loss: 0.3950 - accuracy: 0.8557 - val_loss: 0.3449 - val_accuracy: 0.8768
Epoch 6/40
60000/60000 [==============================] - 5s 82us/sample - loss: 0.3745 - accuracy: 0.8662 - val_loss: 0.3415 - val_accuracy: 0.8754
Epoch 7/40
60000/60000 [==============================] 

In [0]:
# savePath = '/content/drive/My Drive/4th Year/4YP/Fashion MNIST 60k/Results/' + folder + '/R(M,D+).csv'
# pd.DataFrame(RawData_3).to_csv(savePath,header=None, index=None)
print('R(M,D+):',(sum(Correctness_3) / 10000))

R(M,D+): 0.8939


In [0]:
# Test 4: R(M+,D+)
# Full data import, learning and test of: D_train and D_test
# loading modified Fashion MNIST dataset
train = pd.read_csv(pathDplustrain,header=None)
test = pd.read_csv(pathDplustest,header=None)
(n_train,pixels) = np.shape(train)
n_test = np.size(test,0)
dim = int(np.sqrt(pixels-1))

# Getting data in right form
# input image dimensions
img_rows, img_cols = dim, dim

# Extracting data
test_labels = test.iloc[:,0]
test_values = test.iloc[:,1:]
train_labels = train.iloc[:,0]
train_values = train.iloc[:,1:]
x_train = train_values.values.reshape(n_train,img_rows,img_cols,1)
x_test = test_values.values.reshape(n_test,img_rows,img_cols,1)

# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(train_labels, num_classes)
y_test = keras.utils.to_categorical(test_labels, num_classes)

# Fit model and learn
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

# Make predictions
predictions_4 = model.predict(x_test, batch_size=batch_size,
                            verbose=1,)
# Forming into required raw data format
predictions_4 = predictions_4.round(0)
MLresults_4 = np.argmax(predictions_4,axis=1)

Correctness_4 = np.zeros(len(predictions_4))
for i in range(len(predictions_4)):
  if np.array_equal(MLresults_4[i],test_labels[i]):
    Correctness_4[i] = True
  else:
    Correctness_4[i] = False

GroundTruth_4 = test_labels

RawData_4 = np.zeros((n_test,5))
RawData_4[:,1] = GroundTruth_4
RawData_4[:,2] = MLresults_4
RawData_4[:,4] = Correctness_4

for i in range(len(GroundTruth_4)):
  RawData_4[i,0] = i+1

# Saving
savePath = '/content/drive/My Drive/4th Year/4YP/Fashion MNIST 60k/Results/' + folder + '/R(M+,D+).csv'
#pd.DataFrame(RawData_4).to_csv(savePath,header=None, index=None)

# # Testing accuracy for interest
# sum(Correctness_4) / 10000
# #y_test = y_test.astype(np.float32)
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test accuracy:', score[1])



Train on 60000 samples, validate on 10000 samples
Epoch 1/40
60000/60000 [==============================] - 5s 84us/sample - loss: 0.2519 - accuracy: 0.9166 - val_loss: 0.3000 - val_accuracy: 0.8989
Epoch 2/40
60000/60000 [==============================] - 5s 83us/sample - loss: 0.2382 - accuracy: 0.9189 - val_loss: 0.3143 - val_accuracy: 0.8969
Epoch 3/40
60000/60000 [==============================] - 5s 84us/sample - loss: 0.2365 - accuracy: 0.9196 - val_loss: 0.3535 - val_accuracy: 0.8991
Epoch 4/40
60000/60000 [==============================] - 5s 83us/sample - loss: 0.2312 - accuracy: 0.9220 - val_loss: 0.3250 - val_accuracy: 0.8970
Epoch 5/40
60000/60000 [==============================] - 5s 83us/sample - loss: 0.2293 - accuracy: 0.9224 - val_loss: 0.3144 - val_accuracy: 0.8921
Epoch 6/40
60000/60000 [==============================] - 5s 84us/sample - loss: 0.2243 - accuracy: 0.9231 - val_loss: 0.3218 - val_accuracy: 0.8949
Epoch 7/40
60000/60000 [==============================] 

In [0]:
savePath = '/content/drive/My Drive/4th Year/4YP/Fashion MNIST 60k/Results/' + folder + '/R(M+,D+).csv'
pd.DataFrame(RawData_4).to_csv(savePath,header=None, index=None)
print('R(M+,D+):',(sum(Correctness_4) / 10000))

R(M+,D+): 0.8903


In [0]:
# Testing accuracy for interest
print(folder)
print('Accuracies for each test:')
print('R(M,D):',(sum(Correctness_1) / 10000))
print('R(M+,D):',(sum(Correctness_2) / 10000))
print('R(M,D+):',(sum(Correctness_3) / 10000))
print('R(M+,D+):',(sum(Correctness_4) / 10000))

In [0]:
!kill 725
%tensorboard --logdir logs  # start Tensorboard


In [0]:
# Sets paths for train/test D/D+
folder = 'Rotating'
path = '/content/drive/My Drive/4th Year/4YP/Fashion MNIST 60k/Results/'
pathDM  = path[:74] + folder  + '/R(M,D).csv'
pathDplusM  = path[:74] + folder  + '/R(M,D+).csv'
pathDMplus  = path[:74] + folder  + '/R(M+,D).csv'
pathDplusMplus  = path[:74] + folder  + '/R(M+,D+).csv'

DM = pd.read_csv(pathDM,header=None).to_numpy()
DplusM = pd.read_csv(pathDplusM,header=None).to_numpy()
DMplus = pd.read_csv(pathDMplus,header=None).to_numpy()
DplusMplus = pd.read_csv(pathDplusMplus,header=None).to_numpy()

Correctness_1 = DM[:,4].sum() / 10000
Correctness_2 = DMplus[:,4].sum() / 10000
Correctness_3 = DplusM[:,4].sum() / 10000
Correctness_4 = DplusMplus[:,4].sum() / 10000

print(folder)
print('Accuracies for each test:')
print('R(M,D):',Correctness_1)
print('R(M+,D):',Correctness_2)
print('R(M,D+):',Correctness_3)
print('R(M+,D+):',Correctness_4)

Rotating
Accuracies for each test:
R(M,D): 0.6875
R(M+,D): 0.1737
R(M,D+): 0.2613
R(M+,D+): 0.8782
